In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd 

# Definir el símbolo de ETH y las fechas deseadas
symbol = "ETH-USD"
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2023, 5, 3)

# Obtener los precios de ETH desde Yahoo Finance
data = yf.download(symbol, start=start_date, end=end_date)

#Calculo del RSI comun
ruedas= 14
data['dif']= data['Close'].diff()
data['win']= np.where(data['dif'] > 0, data['dif'], 0)
data['loss']= np.where(data['dif'] < 0, abs(data['dif']), 0)
data['ema_win']= data.win.ewm(alpha=1/ruedas).mean()
data['ema_loss']= data.loss.ewm(alpha=1/ruedas).mean()
data['rs']= data.ema_win / data.ema_loss
data['rsi']= 100 - (100 / (1+data.rs))


# Crear nueva columna para el precio medio
data["precio_medio"] = (data.Open + data.Close + data.Low + data.High) / 4 

# Crear nueva columna para el volumen en millones de dólares
data["vol_mln_usd"] = round((data.Volume * data.precio_medio) / 1000000, 2)

# Dividir el volumen por un millón para obtener un tamaño de punto manejable
vol_size = data['vol_mln_usd'] / 1e6

 #--------------Configuracion de grafico-----------#


# Crear una figura con un tamaño de 20x12 pulgadas
fig = plt.figure(figsize=(25, 17))
#-----------------------------Grafico HeatMap------------------------#


# Crear el primer objeto de ejes para el gráfico de líneas con los precios de cierre y el gráfico de dispersión con tamaños de puntos variables según el volumen y un colormap
ax = fig.add_subplot(311)
ax.plot(data.index, data['Close'], color='orange', label='Precio de cierre de ETH')

#-------HeatMap----------#

sc = ax.scatter(data.index, data['Close'], s=vol_size*10, c=data['rsi'], cmap='viridis', alpha=0.5)

# Ajustar los límites de los ejes para que el gráfico de dispersión aparezca encima del gráfico de líneas
ax.set_ylim([data['Close'].min()*0.95, data['Close'].max()*1.05])
ax.set_xlim([data.index.min(), data.index.max()])

# Agregar etiquetas de los ejes y leyendas
ax.set_xlabel('Fecha')
ax.set_ylabel('Precio de cierre de ETH')
ax.set_title('Precio de cierre de ETH y Volumen y RSI | HEATMAP', pad=100)
ax.legend(loc='upper right')

# Agregar una barra de color para el mapa de color y modificar su tamaño y posición
cbar = plt.colorbar(sc, ax=ax, location="top", shrink=0.35)
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.label.set_color('black')
cbar.ax.xaxis.label.set_size(12)
cbar.ax.set_xlabel('Índice de fuerza relativa - RSI', labelpad=10)


ax.grid()



#-----------------------------Grafico Volumen------------------------#


# Crear el segundo objeto de ejes para el gráfico de barras con el volumen en millones de dólares
ax1 = fig.add_subplot(312)
ax1.bar(data.index, data['vol_mln_usd'], color='blue', alpha=0.5)


# Ajustar los límites de los ejes para que el gráfico de barras aparezca correctamente
ax1.set_ylim([data['vol_mln_usd'].min()*0.95, data['vol_mln_usd'].max()*1.05])
ax1.set_xlim([data.index.min(), data.index.max()])

# Agregar etiquetas de los ejes y leyendas
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Volumen en millones de dólares')
ax1.set_title('Precio de cierre de ETH y volumen en millones de dólares | Sin HEATMAP')
ax1.grid()

# Agregar segunda escala en el eje y para mostrar el precio de cierre
ax2 = ax1.twinx()
ax2.plot(data.index, data['Close'], color='orange', label='Precio de cierre de ETH')
ax2.set_ylim([data['Close'].min()*0.95, data['Close'].max()*1.05])
ax2.legend(loc='upper right')
ax2.set_ylabel('Precio de cierre de ETH')



#-----------------------------Grafico RSI------------------------#


# Agregar RSI con precio de cierre 

ax3= fig.add_subplot(313)
ax3.plot(data.rsi, ls='--', label= 'RSI')

#Etiquetas
ax3.legend(loc='upper right')
ax3.set_title('Precio de cierre de ETH y RSI | Sin HEATMAP')
ax3.set_ylabel('RSI')
ax3.set_xlabel('Fecha')
ax3.grid()

ax4 = ax3.twinx()
ax4.set_xlim(data.index.min(), data.index.max())
ax4.plot(data.Close, c='orange', label='RSI')
ax4.set_ylabel('Precio de cierre de ETH')

# Mostrar el gráfico

plt.show()